In [3]:
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
import os
import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dash.dependencies import Input, Output, State
from pymongo import MongoClient
from bson.json_util import dumps
from animal_shelter import AnimalShelter
from jupyter_plotly_dash import JupyterDash


###########################
# Data Manipulation / Model
###########################

username = "accuser"
password = "Dodson"
shelter = AnimalShelter(username, password)


#class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SNHU CS-340 Dashboard - Clay Dodson')

#Adds in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#Layout for the datatable
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Clay Dodson'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),
        
#Adds in code to use radio buttons for the interactive filtering options.
    html.Div(
            [
                html.B("Step 1: "), "Select a rescue type from the options below:",
                html.Br(),
                dcc.RadioItems(
                    id="radio_items_id",
                    options=[
                        {"label": "Water Rescue", "value": "WR"},
                        {"label": "Mountain or Wilderness Rescue", "value": "MR"},
                        {"label": "Disaster or Individual Rescue", "value": "DR"},
                        {"label": "Reset", "value": "R"},
                    ],
                    # value="R",
                    labelStyle={"display": "inline-block"},
                ),
                html.Br(),
                html.B("Step 2: "), "Click on the circle on the left of the row within the table to filter the plots below. Clicking on a row highlights the dog's name in the bar chart.",
                html.Br(),
            ]
        ),
    html.Div([
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
    )],
    
#Sets up the features for interactive data table to make it user-friendly for client        

    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    
#Sets up the dashboard so chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
    ])
)

#############################################
# Interaction Between Components / Controller
#############################################


#Callbacks for dashboard filtering
def update_dashboard(filter_type):

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    return (data,columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

#Callback for graph
def update_graphs(viewData):
    dff = df if viewData is None else pd.DataFrame(viewData)

    colors = [
        "#7FDBFF" if i in derived_virtual_selected_rows else "#0074D9" for i in range(len(dff))
    ]

    return [
        dcc.Graph(
            id=column,
            figure={
                "data": [{"x": dff["name"], "type": "bar", "marker": {"color": colors},}],
                "layout": {
                    "xaxis": {"automargin": True},
                    "yaxis": {"automargin": True, "title": {"text": column}},
                    "height": 250,
                    "margin": {"t": 10, "l": 10, "r": 10},
                },
            },
        )
        # check if column exists
        for column in ["age_upon_outcome_in_weeks"]
        if column in dff
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
    
#Callback for radio button
def update_datatable(value):
    if value == "R":
        df = pd.DataFrame.from_records(aac.read_all()).to_dict("records")
        print("Reset button pressed")
        return df
    if value == "WR":
        df = pd.DataFrame.from_records(aac.filter_water_rescue())
        print(f"Filtered to Water Rescue \n {df.head(5)}")
        return df.to_dict("records")
    if value == "MR":
        df = pd.DataFrame.from_records(aac.filter_mountain_wilderness())
        print(f"Filtered to Mountain  \n {df.head(5)}")
        return df.to_dict("records")
    if value == "DR":
        df = pd.DataFrame.from_records(aac.filter_disaster_rescue_tracking())
        print(f"Filtered to Disaster Rescue \n {df.head(5)}")
        return df.to_dict("records")

@app.callback(
    Output("datatable_id", "data"), 
    [Input("radio_items_id", "value")])

#Callback for geolocation chart
def update_map(viewData):
    if viewData is None:
        return[]
    else:
    dff = pd.DataFrame.from_dict(viewData)

    # Austin's GPS coordinates are at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'], dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff.loc[0,'breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
                ])
            ])
        ])
    ]

@app.callback(
    Output("datatable-interactivity-map", "children"), 
    [Input("map", "location_lat_lon_acc")])

#Identifies server to run on    
if __name__ == "__main__":
    app.run_server(port=47778)

    

SyntaxError: invalid syntax (<ipython-input-3-4f5a4e448f46>, line 125)